This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with only gain corruptions and a gain-only solver,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from dask.distributed import LocalCluster
from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
# Simple bandpass calibration with internally generated cluster
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, Fint200=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, Fint200=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-17 18:54:19,083 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-17 18:54:19,085 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-17 18:54:19,086 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-17 18:54:19,096 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-17 18:54:19,643 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-17 18:54:19,819 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-17 18:54:20,419 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-17 18:54:20,420 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-17 18:54:21,220 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-17 18:54:21,365 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 5.1 seconds


In [3]:
# Simple bandpass calibration with externally generated cluster
t0 = perf_counter()
cluster = LocalCluster()
print(f"Sending scheduler address {cluster.scheduler_address}")
bandpass_calibration.run(
    {
        "dask_scheduler_address": cluster.scheduler_address,
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, Fint200=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, Fint200=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-17 18:54:24,938 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-17 18:54:24,939 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-17 18:54:24,940 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-17 18:54:24,968 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-17 18:54:25,011 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-17 18:54:25,121 - workflow.utils - INFO - Applying direction-independent gain corruptions


Sending scheduler address tcp://127.0.0.1:34881


2024-12-17 18:54:25,755 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-17 18:54:25,756 - pipeline.bandpass_calibration - INFO - Using existing dask cluster tcp://127.0.0.1:34881
2024-12-17 18:54:25,776 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-17 18:54:25,861 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12-17 18:54:25,886 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2024-12-17 18:54:25,940 - pipeline.bandpass_calibration - INFO - Running graph and returning calibration solutions
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibilit

pipeline ran in 3.6 seconds
